In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly
import scipy.signal as ss
import math
from vectors import Point, Vector
from dataFilter import butter_highpass_filter, butter_lowpass_filter
from selfpkg import readDB, MinToMS

In [2]:
def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

In [3]:
#funcstion for calculate vector related info
def getMag(list):
    ret = math.sqrt(list[0]**2 + list[1]**2 + list[2]**2)
    return ret

def getUnit(list):
    mag = getMag(list);
    ret = [x / mag for x in list]
    return ret

def multiply(list, n):
    ret = [x * n for x in list]
    return ret

def add(list1, list2):
    ret = [list1[0]+list2[0], list1[1]+list2[1], list1[2]+list2[2]]
    return ret

def substract(list1, list2):
    ret = [list1[0]-list2[0], list1[1]-list2[1], list1[2]-list2[2]]
    return ret

def dot(list1, list2):
    ret = list1[0]*list2[0] + list1[1]*list2[1] + list1[2]*list2[2]
    return ret

def cross(list1, list2):
    ret = [
             list1[1]*list2[2] - list1[2]*list2[1], \
             list1[2]*list2[0] - list1[0]*list2[2], \
             list1[0]*list2[1] - list1[1]*list2[0]
            ]
    return ret

In [4]:
def getDirection(df):
    n = len(df.index)
    
    acc = []
    acc_x = df.accX/10;
    acc_y = df.accY/10;
    acc_z = df.accZ/10;
    for i in range(n):
        acc.append([acc_x[i], acc_y[i], acc_z[i]])
    magn_x = df.compassX
    magn_y = df.compassY
    magn_z = df.compassZ
    magn = []
    for i in range(n):
        magn.append([magn_x[i], magn_y[i], magn_z[i]])
        
    #move the downward component of the magnetic field(~40 mT)
    #1st. calculate the downward vector defined by gravity
    unit_G_vector = [] # unit vector @G
    for i in range(n):
        unit_G_vector.append(getUnit(acc[i]))

    magn_downward_dot_product = [] #dot product of unit_G_vector & magn
    for i in range(n):
        magn_downward_dot_product.append(dot(magn[i], unit_G_vector[i]))

    magn_downward = [] #projection of magn on G
    for i in range(n):
        magn_downward.append(multiply(unit_G_vector[i], magn_downward_dot_product[i]))
        
    #2nd. substract the downward component from the total magnetic vector
    #Now we have the south vector
    south_vector = []
    for i in range(n):
        south_vector.append(substract(magn[i], magn_downward[i]))
    
    #3rd. projection of z on horizontal plane
    z_project_vertical = []
    for i in range(n):
        z_project_vertical.append(multiply(unit_G_vector[i], dot([0, 0, 1], unit_G_vector[i])))

    z_in_horizontal = []
    for i in range(n):
        z_in_horizontal.append(substract([0, 0, 1], z_project_vertical[i]))
        
    #4th calculate the angle between the projection of z in horizontal plane & south
    sin = []
    cos = []
    for i in range(n):
        try:
            sin.append(dot(cross(z_in_horizontal[i], south_vector[i]), unit_G_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
            cos.append(dot(z_in_horizontal[i], south_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        except ZeroDivisionError:
            if getMag(z_in_horizontal[i]) == 0:
                z_in_horizontal[i][0] = 0.1
            if getMag(south_vector[i]) == 0:
                south_vector[i][0] = 0.1
            sin.append(dot(cross(z_in_horizontal[i], south_vector[i]), unit_G_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
            cos.append(dot(z_in_horizontal[i], south_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
            continue
    
    #finally, calculate the angle
    angles = []
    for i in range(n):
        try:
            if cos[i] >= 0 and sin[i] >= 0:
                angles.append(math.asin(sin[i]) / 3.14 * 180)
            elif cos[i] >= 0 and sin[i] <= 0:
                angles.append(math.asin(sin[i]) / 3.14 * 180)
            elif cos[i] <= 0 and sin[i] >= 0:
                angles.append(180 - (math.asin(sin[i]) / 3.14 * 180))
            elif cos[i] <= 0 and sin[i] <= 0:
                angles.append(-(math.asin(sin[i]) / 3.14 * 180) - 180)
        
        except ValueError:
            angles.append(0)
            continue
    
    return angles

In [5]:
def getDirectionByRotation(df):
    n = len(df.index)

    acc = []
    acc_x = df.accX/10;
    acc_y = df.accY/10;
    acc_z = df.accZ/10;
    for i in range(n):
        acc.append([acc_x[i], acc_y[i], acc_z[i]])

    rotation_x = df.rotationalSpeedX
    rotation_y = df.rotationalSpeedY
    rotation_z = df.rotationalSpeedZ
    rotation = []
    for i in range(n):
        rotation.append([rotation_x[i], rotation_y[i], rotation_z[i]])

    unit_G_vector = [] # unit vector @G
    for i in range(n):
        unit_G_vector.append(getUnit(acc[i]))

    rotation_project_G = [] #Rg
    for i in range(n):
        rotation_project_G.append(dot(rotation[i], unit_G_vector[i]))

    heading_horizontal = [0]
    for i in range(n-1):
        heading_horizontal.append(heading_horizontal[i] + (rotation_project_G[i+1] * (df.time[i+1]-df.time[i])))
    
    return [x/10000 for x in heading_horizontal]

In [6]:
def getTurningPoint(df):
    #get moving average of angles_rotation
    b = movingaverage(df.angles_rotation, 100)
    #set conv mask & conv
    mask = [-1, -1/2, 0, 1/2, 1] 
    conv = np.convolve(b, mask)
    #< threshold, set zero
    for i in range(len(conv)):
        if abs(conv[i]) < 0.9:
            conv[i] = 0
    #get start & end point of each bend
    i = 0
    j = 0
    index = []
    while(j < len(conv)):
        if conv[i] == 0 and conv[j] == 0:
            j += 1
        elif conv[i] == 0 and conv[j] != 0:
                i = j
                j += 1
        elif conv[i] != 0 and conv[j] != 0:
            j += 1
        elif conv[i] != 0 and conv[j] == 0:
            index.append(i)
            index.append(j)
            i = j
            j += 1
    #get rid of first turning point, because:
    #1. extra part of trim
    #2. shift of low pass
    while (index[1] / len(conv)) < 0.04:
        index = index[2:]

    #get turning angle
    turningAngle = []
    for i in range(len(index)):
        if i % 2 == 0:
            turningAngle.append(b[index[i+1]] - b[index[i]])
            turningAngle.append(b[index[i+1]] - b[index[i]])

    return turningAngle, (df.time[index]/60000).tolist()

In [7]:
run = 4
df = pd.read_csv('/Users/meng/Downloads/Wise_Run' + str(run) + '_Trimed.csv')
# df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/angle.csv')
angles = getDirection(df)
angles = butter_lowpass_filter(angles, 1, 50, 5).tolist() #lowpass
angles_rotation = getDirectionByRotation(df)
angles_rotation = butter_lowpass_filter(angles_rotation, 1, 50, 5).tolist() #lowpass

In [8]:
if len(angles) < len(df.index):
    length = len(df.index) - len(angles)
    for i in range(length):
        angles.append(0)
if len(angles_rotation) < len(df.index):
    length = len(df.index) - len(angles_rotation)
    for i in range(length):
        angles_rotation.append(0)
df['angles'] = 0
df['angles'] = angles
df['angles_rotation'] = 0
df['angles_rotation'] = angles_rotation

In [10]:
df.to_csv('08Angle.csv')

In [11]:
time, angle = getTurningPoint(df)
print(time)
print(angle)

[0.33189424404879997, 0.33189424404879997, -0.893248270213129, -0.893248270213129, 0.699253677699966, 0.699253677699966, -2.5780051237054433, -2.5780051237054433, -42.409814571419076, -42.409814571419076, -14.686830997657097, -14.686830997657097, -45.58690948389348, -45.58690948389348, 18.464008839778145, 18.464008839778145, -78.80916886796024, -78.80916886796024, -6.976753128863123, -6.976753128863123, 77.04365628626684, 77.04365628626684, 1.0866262027496987, 1.0866262027496987]
[3.3109, 3.3116666666666665, 3.8416166666666665, 3.8436166666666667, 3.8863166666666666, 3.8880166666666667, 6.8357833333333335, 6.84035, 6.842833333333333, 6.88595, 6.987733333333333, 7.019166666666667, 7.1877, 7.230533333333334, 8.832783333333333, 8.961183333333333, 9.128033333333333, 9.176733333333333, 9.1784, 9.189233333333334, 14.032966666666667, 14.076466666666667, 15.01465, 15.015916666666667]


In [9]:
trace0 = go.Scattergl(
    x=df.time/60000,
    y=df.angles,
    name = 'angles',
    line = dict(
        color = ('rgb(255, 0, 0)'),
        width = 1)
)

trace1 = go.Scattergl(
    x=df.time/60000,
    y=df.angles_rotation,
    name = 'angles from rotation',
    line = dict(
        color = ('rgb(0, 0, 255)'),
        width = 1)
)

# trace2 = go.Scattergl(
#     x=df.time/60000,
#     y=(df.compassX ** 2 + df.compassY ** 2 + df.compassZ ** 2) ** (1/2),
#     name = 'Magnetic sensor output',
#     line = dict(
#         color = ('rgb(0, 0, 255)'),
#         width = 1)
# )
# a = movingaverage(df.angles, 100)
# b = movingaverage(df.angles_rotation, 100)

# trace3 = go.Scattergl(
#     x=df.time/60000,
#     y=a,
#     name = 'angles avg',
#     line = dict(
#         color = ('rgb(200, 0, 0)'),
#         width = 1
#     )   
# )

# trace4 = go.Scattergl(
#     x=df.time/60000,
#     y=b,
#     name = 'angles rotation avg',
#     line = dict(
#         color = ('rgb(0, 0, 200)'),
#         width = 1
#     )   
# )

# mask = [-1, -1/2, 0, 1/2, 1] 
# conv = np.convolve(b, mask)
# for i in range(len(conv)):
#     if abs(conv[i]) < 0.9:
#         conv[i] = 0
# i = 0
# j = 0
# r = 0
# index = []
# while(j < len(conv)):
#     if conv[i] == 0 and conv[j] == 0:
#         j += 1
#     elif conv[i] == 0 and conv[j] != 0:
# #         if r != 0 and j - i < 15:
# #             i = r
# #             j += 1
# #         else:   
#             i = j
#             j += 1
#     elif conv[i] != 0 and conv[j] != 0:
#         j += 1
#     elif conv[i] != 0 and conv[j] == 0:
#         index.append(i)
#         index.append(j)
# #         r = i
#         i = j
#         j += 1
        
# while (index[1] / len(conv)) < 0.04:
#     index = index[2:]


# turningAngle = []
# for i in range(len(index)):
#     if i % 2 == 0:
#         turningAngle.append(b[index[i+1]] - b[index[i]])
#         turningAngle.append(b[index[i+1]] - b[index[i]])
        
# print(turningAngle)
# print((df.time[index]/60000).tolist())
    
# trace5 = go.Scattergl(
#     x=df.time/60000,
#     y=np.convolve(a, mask),
#     name = 'angles slope',
#     line = dict(
#         color = ('rgb(127, 127, 127)'),
#         width = 1
#     )   
# )

# trace6 = go.Scattergl(
#     x=df.time/60000,
#     y=conv,
#     name = 'angles rotation slope',
#     line = dict(
#         color = ('rgb(127, 127, 127)'),
#         width = 1
#     )   
# )

# trace7 = go.Scattergl(
#     x=df.time[index]/60000,
#     y=[0] * len(index),
#     name = 'test',
#     line = dict(
#         color = ('rgb(255, 0, 0)'),
#         width = 1
#     )   
# )

data = [trace0, trace1] #, trace3, trace4, trace5, trace6, trace7]
layout = dict(title = 'run' + str(run) + 'angles',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'Angles(degrees)'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='run' + str(run) + ' angles' + '.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/run4 angles.html'

# the following is the distributed/non-functional 'getDirection' for test

In [92]:
run = '04'
df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/sa_run' + run + '_trim.csv')

In [93]:
n = len(df.index)

acc = []
acc_x = df.accX/10;
acc_y = df.accY/10;
acc_z = df.accZ/10;
for i in range(n):
    acc.append([acc_x[i], acc_y[i], acc_z[i]])
magn_x = df.compassX
magn_y = df.compassY
magn_z = df.compassZ
magn = []
for i in range(n):
    magn.append([magn_x[i], magn_y[i], magn_z[i]])

#move the downward component of the magnetic field(~40 mT)
#1st. calculate the downward vector defined by gravity
unit_G_vector = [] # unit vector @G
for i in range(n):
    unit_G_vector.append(getUnit(acc[i]))

magn_downward_dot_product = [] #dot product of unit_G_vector & magn
for i in range(n):
    magn_downward_dot_product.append(dot(magn[i], unit_G_vector[i]))

magn_downward = [] #projection of magn on G
for i in range(n):
    magn_downward.append(multiply(unit_G_vector[i], magn_downward_dot_product[i]))

#2nd. substract the downward component from the total magnetic vector
#Now we have the south vector
south_vector = []
for i in range(n):
    south_vector.append(substract(magn[i], magn_downward[i]))

#3rd. projection of z on horizontal plane
z_project_vertical = []
for i in range(n):
    z_project_vertical.append(multiply(unit_G_vector[i], dot([0, 0, 1], unit_G_vector[i])))

z_in_horizontal = []
for i in range(n):
    z_in_horizontal.append(substract([0, 0, 1], z_project_vertical[i]))

#4th calculate the angle between the projection of z in horizontal plane & south
sin_horizontal = []
cos_horizontal = []
sin_vertical = []
cos_vertical = []
for i in range(n):
    try:
        sin_horizontal.append(dot(cross(z_in_horizontal[i], south_vector[i]), unit_G_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        cos_horizontal.append(dot(z_in_horizontal[i], south_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        
        sin_vertical.append(dot(cross([0, 0, 1], unit_G_vector[i]), getUnit(z_in_horizontal[i])) / (getMag([0, 0, 1]) * getMag(unit_G_vector[i])))
        cos_vertical.append(dot([0, 0, 1], unit_G_vector[i]) / (getMag([0, 0, 1]) * getMag(unit_G_vector[i])))
    except ZeroDivisionError:
        if getMag(z_in_horizontal[i]) == 0:
            z_in_horizontal[i][0] = 0.1
        if getMag(z_project_vertical[i]) == 0:
            z_project_vertical[i][0] = 0.1
        if getMag(south_vector[i]) == 0:
            south_vector[i][0] = 0.1
        sin_horizontal.append(dot(cross(z_in_horizontal[i], south_vector[i]), unit_G_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        cos_horizontal.append(dot(south_vector[i], z_in_horizontal[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        
        sin_vertical.append(dot(cross([0, 0, 1], unit_G_vector[i]), unit_G_vector[i]) / (getMag([0, 0, 1]) * getMag(unit_G_vector[i])))
        cos_vertical.append(dot([0, 0, 1], unit_G_vector[i]) / (getMag([0, 0, 1]) * getMag(unit_G_vector[i])))
        continue

#finally, calculate the angle
angles_horizontal = []
angles_vertical = []
for i in range(n):
    try:
        if cos_horizontal[i] >= 0 and sin_horizontal[i] >= 0:
            angles_horizontal.append(math.asin(sin_horizontal[i]) / 3.14 * 180)
        elif cos_horizontal[i] >= 0 and sin_horizontal[i] <= 0:
            angles_horizontal.append(math.asin(sin_horizontal[i]) / 3.14 * 180)
        elif cos_horizontal[i] <= 0 and sin_horizontal[i] >= 0:
            angles_horizontal.append(180 - (math.asin(sin_horizontal[i]) / 3.14 * 180))
        elif cos_horizontal[i] <= 0 and sin_horizontal[i] <= 0:
            angles_horizontal.append(-(math.asin(sin_horizontal[i]) / 3.14 * 180) - 180)

    except ValueError:
        angles_horizontal.append(0)
        continue
        
for i in range(n):
    try:
        angles_vertical.append(math.acos(cos_vertical[i]) / 3.14 * 180)
#         if cos_vertical[i] >= 0 and sin_vertical[i] >= 0:
#             angles_vertical.append(math.asin(sin_vertical[i]) / 3.14 * 180)
#         elif cos_vertical[i] >= 0 and sin_vertical[i] <= 0:
#             angles_vertical.append(math.asin(sin_vertical[i]) / 3.14 * 180)
#         elif cos_vertical[i] <= 0 and sin_vertical[i] >= 0:
#             angles_vertical.append(180 - (math.asin(sin_vertical[i]) / 3.14 * 180))
#         elif cos_vertical[i] <= 0 and sin_vertical[i] <= 0:
#             angles_vertical.append(-(math.asin(sin_vertical[i]) / 3.14 * 180) - 180)

    except ValueError:
        angles_vertical.append(0)
        continue

In [94]:
trace0 = go.Scattergl(
    x=df.time/60000,
    y=angles_horizontal,
    name = 'horizontal',
    line = dict(
        color = ('rgb(255, 0, 0)'),
        width = 1)
)

trace2 = go.Scattergl(
    x=df.time/60000,
    y=angles_vertical,
    name = 'vertical',
    line = dict(
        color = ('rgb(0, 0, 255)'),
        width = 1)
)

trace3 = go.Scattergl(
    x=df.time/60000,
    y=[90] * len(df.index),
    name='90 degree',
    line = dict(
        color = ('rgb(127, 127, 127)'),
        width = 1
    )
)

data = [trace0, trace2, trace3]
layout = dict(title = 'run' + str(run) + 'angles',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'Angles(degrees)'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='run' + str(run) + ' angles' + '.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/run06 angles.html'

In [54]:
unit_heading_vector = [] # unit vector @heading direction
# for i in range(n):
#     unit_heading_vector.append(getUnit(z_in_horizontal[i]))
for i in range(n):
    unit_heading_vector.append([0, 0, 1])

acc_project_horizontal = []
for i in range(n):
    acc_project_horizontal.append(dot(acc[i], unit_heading_vector[i]))

velocity_horizontal = [acc_project_horizontal[0] * 0.02 * 3.2808399]
for i in range(n-1):
        velocity_horizontal.append(velocity_horizontal[i] + (acc_project_horizontal[i+1] * (df.time[i+1] - df.time[i])/1000 * 3.2808399))

In [65]:
slope_velocity = [0]
for i in range(n-1):
    slope_velocity.append((velocity_horizontal[i+1] - velocity_horizontal[i]) / ((df.time[i+1] - df.time[i])/1000) * 3.2808399)
    
avg = movingaverage(slope_velocity, 500)
# for i in range(n-1):
#     if abs(slope_velocity[i] - avg[i]) > 10:
#         slope_velocity[i] = avg[i]        

In [67]:
trace0 = go.Scattergl(
    x=df.time/60000,
    y=[x for x in velocity_horizontal],
    name = 'velocity',
    line = dict(
        color = ('rgb(255, 0, 0)'),
        width = 1)
)

trace1 = go.Scattergl(
    x=df.time/60000,
    y=slope_velocity,
    name = 'slope',
    line = dict(
        color = ('rgb(127, 127, 127)'),
        width = 1)
)

trace2 = go.Scattergl(
    x=df.time/60000,
    y=avg,
    name = 'avg',
    line = dict(
        color = ('rgb(0, 0, 255)'),
        width = 1)
)


data = [trace0, trace1, trace2]
layout = dict(title = 'run08 velocity',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'velocity(ft/s)'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='run08 velocity' + '.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/run08 velocity.html'

In [9]:
trace0 = go.Scattergl(
    x=df.time/60000,
    y=butter_lowpass_filter(angles_horizontal, 1, 50, 5),
#     y=angles_horizontal,
    name = 'horizontal angles',
    line = dict(
        color = ('rgb(255, 0, 0)'),
        width = 1)
)

trace1 = go.Scattergl(
    x=df.time/60000,
    y=angles_vertical,
    name = 'vertical angles',
    line = dict(
        color = ('rgb(127, 127, 127)'),
        width = 1)
)

trace2 = go.Scattergl(
    x=df.time/60000,
    y=[getMag(x) for x in magn],
    name = 'magnetic magnitude',
    line = dict(
        color = ('rgb(0, 0, 255)'),
        width = 1)
)

data = [trace0, trace2, trace1]
layout = dict(title = 'run04 angle',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'angles(degrees)'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='run04 angle' + '.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/run04 angle.html'

In [55]:
if len(angles_horizontal) < len(df.index):
    length = len(df.index) - len(angles_horizontal)
    for i in range(length):
        angles_horizontal.append(0)
df['angles'] = 0
df['angles'] = angles_horizontal
df.to_csv('run06 with angles.csv')

In [80]:
trace0 = go.Scattergl(
    x=df.time/60000,
    y=butter_lowpass_filter(angles, 1, 50, 5),
#     y=angles_horizontal,
    name = 'horizontal angles',
    line = dict(
        color = ('rgb(255, 0, 0)'),
        width = 1)
)

trace1 = go.Scattergl(
    x=df.time/60000,
    y=butter_lowpass_filter(angles_rotation, 1, 50, 5),
    name = 'rotation horizontal',
    line = dict(
        color = ('rgb(127, 127, 127)'),
        width = 1)
)

# trace2 = go.Scattergl(
#     x=df.time/60000,
#     y=[getMag(x) for x in magn],
#     name = 'magnetic magnitude',
#     line = dict(
#         color = ('rgb(0, 0, 255)'),
#         width = 1)
# )

data = [trace0, trace1]
layout = dict(title = 'run04 angle',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'angles(degrees)'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='run04 angle' + '.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/run04 angle.html'

In [86]:
if len(angles) < len(df.index):
    length = len(df.index) - len(angles)
    for i in range(length):
        angles.append(0)
if len(angles_rotation) < len(df.index):
    length = len(df.index) - len(angles_rotation)
    for i in range(length):
        angles_rotation.append(0)
# df['angles'] = 0
# df['angles'] = heading_horizontal
# df['angles'] = df['angles'] / 10000
# df.to_csv('run04 with angles.csv')

In [74]:
# trace0 = go.Scattergl(
#     x=df.time/60000,
#     y=butter_lowpass_filter(angles_horizontal, 1, 50, 5),
# #     y=angles_horizontal,
#     name = 'horizontal angles',
#     line = dict(
#         color = ('rgb(255, 0, 0)'),
#         width = 1)
# )

trace1 = go.Scattergl(
    x=df.time/60000,
    y=[x/10000 for x in heading_horizontal],
    name = 'rotation horizontal',
    line = dict(
        color = ('rgb(127, 127, 127)'),
        width = 1)
)

# trace2 = go.Scattergl(
#     x=df.time/60000,
#     y=[getMag(x) for x in magn],
#     name = 'magnetic magnitude',
#     line = dict(
#         color = ('rgb(0, 0, 255)'),
#         width = 1)
# )

data = [trace1]
layout = dict(title = 'run04 angle',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'angles(degrees)'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='run04 angle' + '.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/run04 angle.html'

In [253]:
# df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/sa_run04_trim.csv')
csv_database = 'sqlite:///sa_run08.db' 
df = readDB(csv_database)

In [254]:
n = len(df.index)

In [255]:
#calculate the robot tilt from acceleration
acc_x = df.accX / 10
acc_y = df.accY /10
acc_z = df.accZ / 10
angle_G_in_xy = []
# acc_xy = []
# angle_G_in_vertical = []
# for x in acc_y/acc_x:
#     angle_G_in_xy.append(math.atan(x) / 3.14 * 180)
    
# for i in range(n):
#     acc_xy.append(math.sqrt(acc_x[i]**2 + acc_y[i]**2))

# for i in range(n):
#     angle_G_in_vertical.append(math.atan(acc_z[i]/acc_xy[i]) / 3.14 * 180)

In [256]:
acc = []
for i in range(n):
    acc.append([acc_x[i], acc_y[i], acc_z[i]])
magn_x = df.compassX
magn_y = df.compassY
magn_z = df.compassZ
magn = []
for i in range(n):
    magn.append([magn_x[i], magn_y[i], magn_z[i]])

In [257]:
#move the downward component of the magnetic field(~40 mT)
#1st. calculate the downward vector defined by gravity
unit_G_vector = [] # unit vector @G
for i in range(n):
    unit_G_vector.append(getUnit(acc[i]))
    
magn_downward_dot_product = [] #dot product of unit_G_vector & magn
for i in range(n):
    magn_downward_dot_product.append(dot(magn[i], unit_G_vector[i]))

magn_downward = [] #projection of magn on G
for i in range(n):
    magn_downward.append(multiply(unit_G_vector[i], magn_downward_dot_product[i]))

In [258]:
#2nd. substract the downward component from the total magnetic vector
#Now we have the south vector
south_vector = []
for i in range(n):
    south_vector.append(substract(magn[i], magn_downward[i]))

In [261]:
#3rd. projection of z on horizontal plane
z_project_vertical = []
for i in range(n):
    z_project_vertical.append(multiply(unit_G_vector[i], dot([0, 0, 1], unit_G_vector[i])))

z_in_horizontal = []
for i in range(n):
    z_in_horizontal.append(substract([0, 0, 1], z_project_vertical[i]))

cos_ZG = []
for i in range(n):
    cos_ZG.append(dot(z_project_vertical[i], unit_G_vector[i]) / (getMag(z_project_vertical[i]) * getMag(unit_G_vector[i])))

/Users/meng/Documents/dataPreprocessing/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in double_scalars



In [263]:
timeConversionVal = 60000
trace0 = go.Scattergl(
    x=df.time/timeConversionVal,
    y=cos_ZG,
    name = 'accX',
    line = dict(
        color = ('rgb(0, 0, 255)'),
        width = 1)
)

data = [trace0]

layout = dict(title = '3',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'Acceleration'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='3' + '.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/3.html'

In [242]:
#4th calculate the angle between the projection of z in horizontal plane & south
sin = []
cos = []
crossp = []
for i in range(n):
    try:
#         crossp.append(cross(z_in_horizontal[i], south_vector[i]))
        sin.append(dot(cross(z_in_horizontal[i], south_vector[i]), unit_G_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        cos.append(dot(z_in_horizontal[i], south_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
    except ZeroDivisionError:
        if getMag(z_in_horizontal[i]) == 0:
            z_in_horizontal[i][0] = 0.1
        if getMag(south_vector[i]) == 0:
            south_vector[i][0] = 0.1
#         crossp.append(cross(z_in_horizontal[i], south_vector[i]))
        sin.append(dot(cross(z_in_horizontal[i], south_vector[i]), unit_G_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        cos.append(dot(z_in_horizontal[i], south_vector[i]) / (getMag(z_in_horizontal[i]) * getMag(south_vector[i])))
        continue

In [197]:
angles = []
for i in range(n):
    try:
        a1 = math.acos(cos[i]) / 3.14 * 180
        a2 = math.asin(sin[i]) / 3.14 * 180
        if (abs(a1 - a2) < 0.1):
            angles.append((a1 + a2)/2)
        else:
            angles.append(-a2)
    except ValueError:
        angles.append(0)
        continue

In [194]:
print(math.acos(cos[0]) / 3.14 * 180)
print(math.asin(sin[0]) / 3.14 * 180)

163.9464472092234
16.144851404204267


In [177]:
print(max(sin))
print(min(sin))
print(max(cos))
print(min(cos))

1.0
1.5613162523842676e-16
1.0000000000000002
-0.9999587934734988
